In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# create a directory to move files

!mkdir /kaggle/working/aerialimage

In [ ]:
import os
import shutil
import matplotlib.pyplot as plt

import numpy as np
import random

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import EarlyStopping

from keras import backend as K

In [ ]:
# create a folders to move train and validation files

!mkdir /kaggle/working/aerialimage/train_input
!mkdir /kaggle/working/aerialimage/train_target

!mkdir /kaggle/working/aerialimage/val_input
!mkdir /kaggle/working/aerialimage/val_target

In [ ]:
# create a folders to move train and validation files

!mkdir /kaggle/working/aerialimage/train_input/input_data
!mkdir /kaggle/working/aerialimage/train_target/target_data

!mkdir /kaggle/working/aerialimage/val_input/input_data
!mkdir /kaggle/working/aerialimage/val_target/target_data

In [ ]:
# move input images to the folder

org_train_input = '/kaggle/input/aerialimagedataset/AerialImageDataset/train/images/'
new_train_input = '/kaggle/working/aerialimage/train_input/input_data'
new_val_input = '/kaggle/working/aerialimage/val_input/input_data'

input_img_paths = []

for dirname, _, filenames in os.walk(org_train_input):
    for filename in filenames:
        input_img_paths.append(os.path.join(dirname, filename))

input_img_paths = sorted(input_img_paths)

random.Random(23).shuffle(input_img_paths)
    
input_img_paths[0:2]


In [ ]:
# copy 40 input images to validation folder

val_num = 40

for file in input_img_paths[:val_num]:
    shutil.copy(file, new_val_input)
    
for file in input_img_paths[val_num:]:
    shutil.copy(file, new_train_input)

In [ ]:
# move target images to the folder

org_train_target = '/kaggle/input/aerialimagedataset/AerialImageDataset/train/gt/'
new_train_target = '/kaggle/working/aerialimage/train_target/target_data'
new_val_target = '/kaggle/working/aerialimage/val_target/target_data'

target_img_paths = []

for dirname, _, filenames in os.walk(org_train_target):
    for filename in filenames:
        target_img_paths.append(os.path.join(dirname, filename))

target_img_paths = sorted(target_img_paths)

random.Random(23).shuffle(target_img_paths)

target_img_paths[0:2]

In [ ]:
# copy 40 target images to validation folder

for file in target_img_paths[:val_num]:
    shutil.copy(file, new_val_target)
    
for file in target_img_paths[val_num:]:
    shutil.copy(file, new_train_target)

In [ ]:
# check whether folder names in input and target folders match

new_train_target = '/kaggle/working/aerialimage/train_target/target_data'
new_val_target = '/kaggle/working/aerialimage/val_target/target_data'

new_train_input = '/kaggle/working/aerialimage/train_input/input_data'
new_val_input = '/kaggle/working/aerialimage/val_input/input_data'

i = 0

for dirname, _, filenames in os.walk(new_train_input):
    for filename in filenames:
        print(filename)
        i += 1
        if i == 5:
            break
        

k = 0

for dirname, _, filenames in os.walk(new_train_target):
    for filename in filenames:
        print(filename)
        k += 1
        if k == 5:
            break

In [ ]:
# create an image generator / data augmentation pipeline 
# augment data in train dataset only

BATCH_SIZE = 5
TARGET_SIZE = (512,512)
seed = 23

train_datagen = ImageDataGenerator(samplewise_std_normalization=False, 
                                   horizontal_flip = True, 
                                   vertical_flip = False, 
                                   height_shift_range = 0.1, 
                                   width_shift_range = 0.1, 
                                   rotation_range = 3, 
                                   shear_range = 0.01,
                                   fill_mode = 'nearest',
                                   zoom_range = 0.05, 
                                   zca_whitening = True,
                                   zca_epsilon=1e-5, 
                                   )


target_datagen = ImageDataGenerator(rescale=1./255,
                                    samplewise_std_normalization=False, 
                                    horizontal_flip = True, 
                                    vertical_flip = False, 
                                    height_shift_range = 0.1, 
                                    width_shift_range = 0.1, 
                                    rotation_range = 3, 
                                    shear_range = 0.01,
                                    fill_mode = 'nearest',
                                    zoom_range = 0.05,
                                    zca_whitening = True,
                                    zca_epsilon=1e-5, 
                                   )

val_input_datagen = ImageDataGenerator()

val_target_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# use data augmentation pipeline and start loading train data from folders

train_generator_input = train_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/train_input',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed = seed

)

train_generator_output = target_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/train_target',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    color_mode="grayscale",
    seed = seed
)

train_generator = zip(train_generator_input, train_generator_output)

In [ ]:
# use data augmentation pipeline and start loading validation data from folders

val_generator_input = val_input_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/val_input',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed = seed

)

val_generator_output = val_target_datagen.flow_from_directory(
    '/kaggle/working/aerialimage/val_target',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode=None,
    color_mode="grayscale",
    seed = seed
)

val_generator = zip(val_generator_input, val_generator_output)

In [ ]:
# define architecture of VGG19 model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same",kernel_initializer = 'he_normal')(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same", kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_vgg19_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained VGG19 Model """
    vgg19 = VGG19(include_top=False, weights="/kaggle/input/vgg19weights/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5", input_tensor=inputs)

    """ Encoder """
    s1 = vgg19.get_layer("block1_conv2").output         ## (512 x 512)
    s2 = vgg19.get_layer("block2_conv2").output         ## (256 x 256)
    s3 = vgg19.get_layer("block3_conv4").output         ## (128 x 128)
    s4 = vgg19.get_layer("block4_conv4").output         ## (64 x 64)

    """ Bridge """
    b1 = vgg19.get_layer("block5_conv4").output         ## (32 x 32)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)

    """ Output """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="VGG19_U-Net")
    return model

In [ ]:
# load the model and output architecture

input_shape = TARGET_SIZE + (3,)
model = build_vgg19_unet(input_shape)
model.summary()

In [ ]:
# compile the model and set up the callback to save the model as we train

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.compile(optimizer="adam",loss='mean_squared_error', metrics=['accuracy'])

callbacks=[keras.callbacks.ModelCheckpoint("VGG19_5DEC2022.keras", save_best_only = True)]

In [ ]:
history = model.fit(train_generator, 
                    epochs=100, 
                    callbacks=callbacks,
                    validation_data=val_generator,
                    steps_per_epoch = 2*140/BATCH_SIZE,
                    validation_steps = 40/BATCH_SIZE)